In [120]:
from collections import Counter
from icecream import ic
import pandas as pd 
import numpy as np

In [207]:
mock_data = {
    'gender':['F','F','F','F','M','M','M'],
    'income':['+10','-10','+10','+10','+10','+10','-10'],
    'family_number':[1, 1, 2, 1, 1, 1, 2],
    'bought':[1, 1, 1, 0, 0, 0, 1,]
}

dataset = pd.DataFrame.from_dict(mock_data)
print(dataset)

#混乱程度
def entropy(elements):
    counter = Counter(elements)
    probs = [counter[c] / len(elements) for c in set(elements)]
#    ic(probs)
    return -np.sum(np.fromiter(((p * np.log(p)) for p in probs),float))

#筛选一个特征
def find_the_optimal_spliter(training_data:pd.DataFrame, target:str) ->str:
    x_fields = set(training_data.columns.tolist()) - {target}
    
    spliter = None
    min_entropy = float('inf') #正无穷（负无穷：float('-inf'）)
    
    for f in x_fields:
        values = set(training_data[f]) #eg. f:'gender' --> values = {'M', 'F'}
        i = 0
        val = []
        for v in values:
            #按f列中的v元素，筛列出 对应的 目的（target）列的 元素（0,1） 的列表
            sub_spliter_1 = training_data[training_data[f] == v][target].tolist()
            entropy_1 = entropy(sub_spliter_1)
            
            val.append(v)
            i += 1
            
            sub_spliter_2 = training_data[training_data[f] != v][target].tolist()
            entropy_2 = entropy(sub_spliter_2)
            
            entropy_v = entropy_1 + entropy_2
            
            if entropy_v < min_entropy:
                min_entropy = entropy_v
                spliter = (f,v)
                
            # 选择混乱程度大的元素类，作为下一次特征选取的调用值   
            if entropy_v == min_entropy:
                if entropy_1 > entropy_2:
                    v = val[i-1]
                    spliter = (f,v)
                                       
#    print('spliter is: {}'.format(spliter))
#    print('the min_entropy is: {}'.format(min_entropy))
    
    return spliter

#筛选次特征
def find_the_1and2_optimal_spliter():
    s_1 = find_the_optimal_spliter(training_data=dataset,target='bought')
    s_2 = find_the_optimal_spliter(dataset[dataset[s_1[0]]==s_1[1]],'bought')
    return s_1 + s_2


# 预测函数
def predict(gender, income, family_number):
    sp = find_the_1and2_optimal_spliter() #如('income', '+10', 'family_number', 1)
    print('the two_spliters are {}'.format(sp))
    
    get = {'gender':gender,'income':income,'family_number':family_number} # 存储
    
    output_1 = 'the outcome of prediction is 1'
    output_0 = 'the outcome of prediction is 0'
        
    end = dataset[dataset[sp[2]] == sp[3]]['bought'].tolist() 
    ran = float(Counter(end)[0] / len(end)) #经过两个特征筛选后，输出为0的概率
    ran_ = np.random.random() # 0到1 之间的随机浮点数
    
    #决策过程
    if get[sp[0]] == sp[1] and get[sp[2]] == str(sp[3]) and ran_ < ran:
        return 0
    else:
        return 1


  gender income  family_number  bought
0      F    +10              1       1
1      F    -10              1       1
2      F    +10              2       1
3      F    +10              1       0
4      M    +10              1       0
5      M    +10              1       0
6      M    -10              2       1


In [211]:
predict('M','-10','1')

the two_spliters are ('income', '+10', 'family_number', 1)


1